<a href="https://colab.research.google.com/github/Peter-Hou/Web-Scrapping-CPSO/blob/main/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from string import ascii_lowercase
import re
import time
from random import randint
import csv
import sys
import os

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from string import ascii_lowercase
import re
import time
from random import randint
import csv
import sys
import os

url_search = "https://doctors.cpso.on.ca/?search=general"
url_paging = "https://doctors.cpso.on.ca//Doctor-Search-Results?type=name&term="



def scrape_doctors(soup):
    new_docs = {}
    for doctor in soup.find_all(class_ = 'doctor-search-results--result'):
        doc_str = doctor.find('h3').text
        new_docs[ (re.search("\d+", doc_str).group()) ] = doctor
    return new_docs

headers_search = {
    "Host": "www.cpso.on.ca",
    "User-Agent": "Adam Kowalczewski, Service Canada, adam.kowalczewski@servicecanada.gc.ca",
    "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8",
    "Accept-Language": "en-US,en;q=0.5",
    "Accept-Encoding": "gzip, deflate, br",
    "Referer": "https://www.cpso.on.ca/Public-Information-Services/Find-a-Doctor?search=general",
    "Content-Type": "application/x-www-form-urlencoded",
    "Connection": "keep-alive",
    "Cookie": "CMSPreferredCulture=en-CA; _ga=GA1.3.788028045.1540596057; _gid=GA1.3.1509832542.1540596057; CMSCsrfCookie=PB+DjsX3FE9SCgAk3jGZzc65Ld6NEaE755HpaDB9; ASP.NET_SessionId=pek14izhwnw4u0curdgkxygh; _gat_UA-36725164-1=1",
    "Upgrade-Insecure-Requests": "1"
}

headers_paging = {
    "Referer": "https://www.cpso.on.ca/Public-Register-Info-(1)/Doctor-Search-Results"
}

manScript = "%3b%3bAjaxControlToolkit%2c+Version%3d4.1.60919.0%2c+Culture%3dneutral%2c+PublicKeyToken%3d28f01b0e84b6d53e%3aen-CA%3aee051b62-9cd6-49a5-87bb-93c07bc43d63%3a475a4ef5%3aeffe2a26%3a7e63a579"
manScript = manScript.replace( "%3b", ";" )
manScript = manScript.replace( "%2c", "," )
manScript = manScript.replace( "%3d", "=" )
manScript = manScript.replace( "%3a", ":" )
manScript = manScript.replace( "en-CA", "en-US")

def crawl_cpso( city_code = '', fsa = '', char = '' ):
    doctors = {}

    # get HTML to generate POST data
    with requests.Session() as s:
        r = s.get(url_search)
        soup = BeautifulSoup(r.content, 'html.parser')

        payload = {
            "__CMSCsrfToken": soup.find("input", id="__CMSCsrfToken")['value'],
            "__VIEWSTATE": soup.find("input", id="__VIEWSTATE")['value'],
            "__VIEWSTATEGENERATOR": soup.find("input", id="__VIEWSTATEGENERATOR")['value'],
            "lng": 'en-CA',
            "manScript_HiddenField": manScript,
            "searchType":"general",
            "p$lt$ctl04$pageplaceholder$p$lt$ctl02$AllDoctorsSearch$advancedState":"closed",
            "p$lt$ctl04$pageplaceholder$p$lt$ctl02$AllDoctorsSearch$ConcernsState":"closed",
            "p$lt$ctl04$pageplaceholder$p$lt$ctl02$AllDoctorsSearch$ddCity": city_code,
            "p$lt$ctl04$pageplaceholder$p$lt$ctl02$AllDoctorsSearch$txtPostalCode": fsa,
            "p$lt$ctl04$pageplaceholder$p$lt$ctl02$AllDoctorsSearch$txtLastName": char,
            "p$lt$ctl04$pageplaceholder$p$lt$ctl02$AllDoctorsSearch$grpGender":"+",
            "p$lt$ctl04$pageplaceholder$p$lt$ctl02$AllDoctorsSearch$grpDocType":"rdoDocTypeAll",
            "p$lt$ctl04$pageplaceholder$p$lt$ctl02$AllDoctorsSearch$ddHospitalName":"-1",
            "p$lt$ctl04$pageplaceholder$p$lt$ctl02$AllDoctorsSearch$ddLanguage":"08",
            "p$lt$ctl04$pageplaceholder$p$lt$ctl02$AllDoctorsSearch$chkActiveDoctors":"on",
            "p$lt$ctl04$pageplaceholder$p$lt$ctl02$AllDoctorsSearch$chkPracticeRestrictions":"on",
            "p$lt$ctl04$pageplaceholder$p$lt$ctl02$AllDoctorsSearch$chkPendingHearings":"on",
            "p$lt$ctl04$pageplaceholder$p$lt$ctl02$AllDoctorsSearch$chkPastHearings":"on",
            "p$lt$ctl04$pageplaceholder$p$lt$ctl02$AllDoctorsSearch$chkHospitalNotices":"on",
            "p$lt$ctl04$pageplaceholder$p$lt$ctl02$AllDoctorsSearch$chkConcerns":"on",
            "p$lt$ctl04$pageplaceholder$p$lt$ctl02$AllDoctorsSearch$chkNotices":"on",
            "p$lt$ctl04$pageplaceholder$p$lt$ctl02$AllDoctorsSearch$btnSubmit1":"Submit"
        }

        # send POST request
        time.sleep(randint(1,3))
        r = s.post( url_search, data = payload )
        soup = BeautifulSoup(r.content, 'html.parser')
        doctors.update( scrape_doctors(soup) )

        while True:
            # stop if there's no pages
            try:
                n_in_group = len(soup.find('div', class_ = "doctor-search-paging").find_all('a', id = re.compile("rptPages")))
            except:
                break

            # page through the rest of the group
            for i in range(1, n_in_group):
                payload_paging = {
                    "__CMSCsrfToken": soup.find("input", id="__CMSCsrfToken")['value'],
                    "__VIEWSTATE": soup.find("input", id="__VIEWSTATE")['value'],
                    "__VIEWSTATEGENERATOR": soup.find("input", id="__VIEWSTATEGENERATOR")['value'],
                    "__EVENTTARGET": "p$lt$ctl04$pageplaceholder$p$lt$ctl03$CPSO_DoctorSearchResults$rptPages$ctl0{:1}$lnbPage",
                    "lng": 'en-CA',
                    "manScript_HiddenField": manScript,
                    "p$lt$ctl04$pageplaceholder$p$lt$ctl03$CPSO_DoctorSearchResults$hdnCurrentPage": 1
                    }
                payload_paging['__EVENTTARGET'] = payload_paging['__EVENTTARGET'].format(i)

                time.sleep(randint(1,3))
                r = s.post( url_paging, headers = headers_search.update(headers_paging), data = payload_paging )
                soup = BeautifulSoup(r.content, 'html.parser')

                #page_num = soup.find('div', class_ = 'doctor-search-count').find('div', class_ = 'text-align--right').text.strip()
                doctors.update( scrape_doctors(soup) )

                payload_paging['p$lt$ctl04$pageplaceholder$p$lt$ctl03$CPSO_DoctorSearchResults$hdnCurrentPage'] += 1

            # stop if there's no more groups
            if soup.find(class_ = "aspNetDisabled next") != None:
                break

            # switch to the next group
            payload_paging['__EVENTTARGET'] = 'p$lt$ctl04$pageplaceholder$p$lt$ctl03$CPSO_DoctorSearchResults$lnbNextGroup'
            time.sleep(randint(1,3))
            r = s.post( url_paging, data = payload_paging )
            soup = BeautifulSoup(r.content, 'html.parser')

    return doctors


def count_doctors( city_code = '', fsa = '', char = '' ):
    with requests.Session() as s:

        r = s.get(url_search)
        soup = BeautifulSoup(r.content, 'html.parser')

        payload = {
            "__CMSCsrfToken": soup.find("input", id="__CMSCsrfToken")['value'],
            "__VIEWSTATE": soup.find("input", id="__VIEWSTATE")['value'],
            "__VIEWSTATEGENERATOR": soup.find("input", id="__VIEWSTATEGENERATOR")['value'],
            "lng": 'en-CA',
            "manScript_HiddenField": manScript,
            "searchType":"general",
            "p$lt$ctl04$pageplaceholder$p$lt$ctl02$AllDoctorsSearch$advancedState":"closed",
            "p$lt$ctl04$pageplaceholder$p$lt$ctl02$AllDoctorsSearch$ConcernsState":"closed",
            "p$lt$ctl04$pageplaceholder$p$lt$ctl02$AllDoctorsSearch$ddCity": city_code,
            "p$lt$ctl04$pageplaceholder$p$lt$ctl02$AllDoctorsSearch$txtPostalCode": fsa,
            "p$lt$ctl04$pageplaceholder$p$lt$ctl02$AllDoctorsSearch$txtLastName": char,
            "p$lt$ctl04$pageplaceholder$p$lt$ctl02$AllDoctorsSearch$grpGender":"+",
            "p$lt$ctl04$pageplaceholder$p$lt$ctl02$AllDoctorsSearch$grpDocType":"rdoDocTypeAll",
            "p$lt$ctl04$pageplaceholder$p$lt$ctl02$AllDoctorsSearch$ddHospitalName":"-1",
            "p$lt$ctl04$pageplaceholder$p$lt$ctl02$AllDoctorsSearch$ddLanguage":"08",
            "p$lt$ctl04$pageplaceholder$p$lt$ctl02$AllDoctorsSearch$chkActiveDoctors":"on",
            "p$lt$ctl04$pageplaceholder$p$lt$ctl02$AllDoctorsSearch$chkPracticeRestrictions":"on",
            "p$lt$ctl04$pageplaceholder$p$lt$ctl02$AllDoctorsSearch$chkPendingHearings":"on",
            "p$lt$ctl04$pageplaceholder$p$lt$ctl02$AllDoctorsSearch$chkPastHearings":"on",
            "p$lt$ctl04$pageplaceholder$p$lt$ctl02$AllDoctorsSearch$chkHospitalNotices":"on",
            "p$lt$ctl04$pageplaceholder$p$lt$ctl02$AllDoctorsSearch$chkConcerns":"on",
            "p$lt$ctl04$pageplaceholder$p$lt$ctl02$AllDoctorsSearch$chkNotices":"on",
            "p$lt$ctl04$pageplaceholder$p$lt$ctl02$AllDoctorsSearch$btnSubmit1":"Submit"
        }

        # get doctor search result count
        time.sleep(randint(1,3))
        r = s.post(url_search, data = payload)
        soup = BeautifulSoup(r.content, 'html.parser')
        target = soup.find('div', class_ = "doctor-search-count").strong.text
        return int(re.search( "\d+", target ).group())


In [83]:
# progess bar
import sys
def progress(count, total, status=''):
    bar_len = 60
    filled_len = int(round(bar_len * count / float(total)))

    percents = round(100.0 * count / float(total), 1)
    bar = '=' * filled_len + '-' * (bar_len - filled_len)

    sys.stdout.write('[%s] %s%s ...%s\r' % (bar, percents, '%', status[0:40].ljust(40)))
    sys.stdout.flush()

In [89]:
for i in range(100):
  progress(i, 100)

In [110]:
def get_phone_and_fax(column_name, value):
    """
    Extracts the phone number or fax number from a given value.

    Args:
        column_name (str): The name of the column ('Phone' or 'Fax') indicating the type of number.
        value (str): The input value that *might*   contains the phone or fax number.

    Returns:
        tuple: A tuple containing the extracted phone or fax number and the modified value with the number removed.

    Notes:
        - The function searches for the specified column_name within the value.
        - If the column_name is found, the function extracts the phone or fax number from the value and removes it from the value.
        - If the column_name is not found, the function returns 'N/A' as the phone or fax number.
    """
    start_index = value.find(column_name)
    end_index = value.find('Fax') if column_name == 'Phone' else len(value)

    if start_index != -1:
        ans = value[start_index + len(column_name) + 1: end_index]
        value = value[: start_index]
        return ans, value
    else:
        return 'N/A', value

In [111]:
def scrape_doctors(soup):
    """
    Extracts data from the given BeautifulSoup object and creates a dataframe.

    Args:
        soup (BeautifulSoup): The BeautifulSoup object representing the parsed HTML.

    Returns:
        pandas.DataFrame: A dataframe with columns based on <h4> and <strong> tags and corresponding values from <p> tags.
    """
    data = []  # List to store extracted data

    for doctor in soup.find_all(class_='doctor-search-results--result'):
        # Extract CPSO value
        doc_str = doctor.find('h3').text
        cpso = re.search("\d+", doc_str).group()

        # Extract name
        name = doc_str.split(" (CPSO")[0]  # Extract name from the <h3> tag

        # Extract <h4> tags and corresponding <p> values
        h4_tags = doctor.find_all('h4')
        p_tags = doctor.find_all('p')

        row_data = {'CPSO': cpso, 'Name': name}  # Create a dictionary for storing data for each row

        #if p_tags

        # Extract data from <h4> and corresponding <p> tags
        for h4, p in zip(h4_tags, p_tags):
            column_name = h4.text.strip(':')  # Remove ':' from the <h4> tag
            value = p.get_text(separator=' ') # Extract the text from <p> tag with space as separator and replace '\xa0' with ' '

            # Create seperate column for comments surrounded by <p> <em>
            if p.find('em'):
              row_data['Comment'] = p.find('em').text
              h4_tags.insert(0, h4)
              continue
            # TODO
            # Include the doctor's former name in the dataset, e.g. cpso = 84891
            # Apply error checking mechanism to detect any unware format of doctors' info

            #print(f"p is {p}") if cpso == "84891" else 0
            #print(f"p previous sibling is {p.find_previous_sibling('h4') }") if cpso == "84891" else 0
            #print(f"h4 is {h4}")if cpso == "84891" else 0
            if p.find_previous_sibling('h4') == h4:
              row_data[column_name] = value
            else:
              h4_tags.insert(0, h4)
              continue

            # Extract Phone Number and Fax Number as separate columns
            phone_col_name = 'Phone'
            fax_col_name = 'Fax'

            if phone_col_name not in row_data:
              # Since phone number can only appear in the first <p> tag, then we avoid running the below code for the condition
              row_data[fax_col_name], value = get_phone_and_fax(fax_col_name, value)
              row_data[phone_col_name], value = get_phone_and_fax(phone_col_name, value)
              #print(row_data[phone_col_name])
              row_data[column_name] = value

        data.append(row_data)  # Add row data to the list=)
    return data


In [115]:
def get_doctor_count(soup):
  row_count_soup = soup.find_all('div', class_ = 'row doctor-search-count')[0]

  total_number_doctors = re.search('\d+', row_count_soup.strong.text).group()

  row_count_all_ps = "".join(str(row_count_soup.find_all('p')))

  page_ind = "1 of"

  start_index = row_count_all_ps.find(page_ind)

  total_pages = re.search('\d+', row_count_all_ps[start_index + len(page_ind) + 1:]).group()

  return int(total_number_doctors), int(total_pages)

In [116]:
url_search = "https://doctors.cpso.on.ca/?search=general"
url_paging = "https://doctors.cpso.on.ca//Doctor-Search-Results?type=name&term="

In [ ]:
import requests
from bs4 import BeautifulSoup
import time
from random import randint
import pandas as pd
import re

doctors = []

# get HTML to generate POST data
with requests.Session() as s:
    r = s.get(url_search)
    soup = BeautifulSoup(r.content, 'html.parser')

    payload = {
      "__CMSCsrfToken": soup.find("input", id="__CMSCsrfToken")['value'],
      "__VIEWSTATE": soup.find("input", id="__VIEWSTATE")['value'],
      "__VIEWSTATEGENERATOR": soup.find("input", id="__VIEWSTATEGENERATOR")['value'],
      "lng": 'en-CA',
      #"manScript_HiddenField": manScript,
      "searchType":"general",
      "p$lt$ctl01$pageplaceholder$p$lt$ctl02$CPSO_AllDoctorsSearch$advancedState:":"closed",
      "p$lt$ctl01$pageplaceholder$p$lt$ctl02$CPSO_AllDoctorsSearch$ConcernsState:":"closed",
      "p$lt$ctl01$pageplaceholder$p$lt$ctl02$CPSO_AllDoctorsSearch$txtLastNameQuick": "",
      "p$lt$ctl01$pageplaceholder$p$lt$ctl02$CPSO_AllDoctorsSearch$txtCPSONumber" : "" ,
      "p$lt$ctl01$pageplaceholder$p$lt$ctl02$CPSO_AllDoctorsSearch$chkActiveDoctors": "on",
      "p$lt$ctl01$pageplaceholder$p$lt$ctl02$CPSO_AllDoctorsSearch$txtCPSONumberGeneral": "",
      "p$lt$ctl01$pageplaceholder$p$lt$ctl02$CPSO_AllDoctorsSearch$txtLastName": "" ,
      "p$lt$ctl01$pageplaceholder$p$lt$ctl02$CPSO_AllDoctorsSearch$ddCity": "",
      "p$lt$ctl01$pageplaceholder$p$lt$ctl02$CPSO_AllDoctorsSearch$txtPostalCode":"",
      "p$lt$ctl01$pageplaceholder$p$lt$ctl02$CPSO_AllDoctorsSearch$grpGender": "M"  ,
      "p$lt$ctl01$pageplaceholder$p$lt$ctl02$CPSO_AllDoctorsSearch$grpDocType": "rdoDocTypeFamly",
      "p$lt$ctl01$pageplaceholder$p$lt$ctl02$CPSO_AllDoctorsSearch$ddHospitalCity": "" ,
      "p$lt$ctl01$pageplaceholder$p$lt$ctl02$CPSO_AllDoctorsSearch$ddHospitalName": "-1",
      "p$lt$ctl01$pageplaceholder$p$lt$ctl02$CPSO_AllDoctorsSearch$ddLanguage": "08",
      "p$lt$ctl01$pageplaceholder$p$lt$ctl02$CPSO_AllDoctorsSearch$chkPracticeRestrictions": "on",
      "p$lt$ctl01$pageplaceholder$p$lt$ctl02$CPSO_AllDoctorsSearch$chkPendingHearings": "on",
      "p$lt$ctl01$pageplaceholder$p$lt$ctl02$CPSO_AllDoctorsSearch$chkPastHearings": "on",
      "p$lt$ctl01$pageplaceholder$p$lt$ctl02$CPSO_AllDoctorsSearch$chkHospitalNotices": "on",
      "p$lt$ctl01$pageplaceholder$p$lt$ctl02$CPSO_AllDoctorsSearch$chkConcerns": "on",
      "p$lt$ctl01$pageplaceholder$p$lt$ctl02$CPSO_AllDoctorsSearch$chkNotices": "on",
      "p$lt$ctl01$pageplaceholder$p$lt$ctl02$CPSO_AllDoctorsSearch$txtExtraInfo": "" ,
      "p$lt$ctl01$pageplaceholder$p$lt$ctl02$CPSO_AllDoctorsSearch$btnSubmit1": "Submit"
  }

    # send POST request for the first page
    time.sleep(randint(1, 3))
    r = s.post(url_search, data=payload)
    soup = BeautifulSoup(r.content, 'html.parser')
    doctors += scrape_doctors(soup)
    df = pd.DataFrame(doctors)

    # get number of doctors satisfy the condition and the number of pages it returns
    doctor_count, page_count = get_doctor_count(soup)

    for i in range(2, page_count + 1):

      # update payload for the second page
      payload["p$lt$ctl01$pageplaceholder$p$lt$ctl03$CPSO_DoctorSearchResults$hdnCurrentPage"] = i

      # send POST request for the second page
      time.sleep(randint(1, 3))
      r = s.post(url_search, data=payload)
      soup = BeautifulSoup(r.content, 'html.parser')
      doctors += scrape_doctors(soup)
      print(f"page {i} finished, there are {len(doctors)} elements in doctors")

    df = pd.DataFrame(doctors)


page 2 finished, there are 20 elements in doctors
page 3 finished, there are 30 elements in doctors
page 4 finished, there are 40 elements in doctors
page 5 finished, there are 50 elements in doctors
page 6 finished, there are 60 elements in doctors
page 7 finished, there are 70 elements in doctors
page 8 finished, there are 80 elements in doctors
page 9 finished, there are 90 elements in doctors
page 10 finished, there are 100 elements in doctors
page 11 finished, there are 110 elements in doctors
page 12 finished, there are 120 elements in doctors
page 13 finished, there are 130 elements in doctors
page 14 finished, there are 140 elements in doctors
page 15 finished, there are 150 elements in doctors
page 16 finished, there are 160 elements in doctors
page 17 finished, there are 170 elements in doctors
page 18 finished, there are 180 elements in doctors
page 19 finished, there are 190 elements in doctors
page 20 finished, there are 200 elements in doctors
page 21 finished, there are 

In [107]:
soup

<!DOCTYPE html PUBLIC "-//W3C//DTD XHTML 1.0 Transitional//EN" "http://www.w3.org/TR/xhtml1/DTD/xhtml1-transitional.dtd">

<html lang="en-CA" xml:lang="en-CA" xmlns="http://www.w3.org/1999/xhtml">
<head id="head"><title>
	CPSO - Doctor Search Results
</title><meta content="text/html; charset=utf-8" http-equiv="content-type"/>
<meta content="no-cache" http-equiv="pragma"/>
<meta content="text/css" http-equiv="content-style-type"/>
<meta content="text/javascript" http-equiv="content-script-type"/>
<!-- ### META ###-->
<meta charset="utf-8"/>
<meta content="width=device-width, initial-scale=1" name="viewport"/>
<meta content="IE=edge" http-equiv="X-UA-Compatible"/>
<link href="/Content/Images/site/favicon.ico" rel="shortcut icon" type="image/x-icon"/>
<!-- ### META - END ###-->
<title>CPSO - Home</title>
<!-- ### FONTS ###-->
<link async="" href="https://fonts.googleapis.com/css?family=Raleway:400,400i,700|Roboto:400,400i,700" rel="stylesheet"/><!-- ROBOTO AND RALEWAY -->
<!-- ### FONTS -